### Databricks will manage the cluster for us.

### organization data is not available in community edition, it is stored in cloud account.
### default storage is blob storage but we can convert it into data lake



# 2. Magic Commands
### %python
### %sql
### %r
### %md
### %fs

# DBFS (databricks file system)
## it is a distributed file system, it creates an abstraction layer.

## Access Data


### service principle, which is kinda application which have the acccess to read the data

##### Entra_id --> app_registration --> if not owned applications --> new registration
##### Certificates & secrets --> new client secret

### Then click on the resource group --> data storage account --> containers --> access control(IAM) --> add new role assignment
##### then search for storage blob daa contributor, then search for service principle name then review+create

# then we need to connect azure data lake storage gen2 using python with data bricks
## [connection details](https://learn.microsoft.com/en-us/azure/databricks/connect/storage/tutorial-azure-storage)
##### service_credential = dbutils.secrets.get(scope="<scope>",key="<service-credential-key>")
##### spark.conf.set("fs.azure.account.auth.type.<storage-account>.dfs.core.windows.net", "OAuth")
##### spark.conf.set("fs.azure.account.oauth.provider.type.<storage-account>.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
##### spark.conf.set("fs.azure.account.oauth2.client.id.<storage-account>.dfs.core.windows.net", "<application-id>")
##### spark.conf.set("fs.azure.account.oauth2.client.secret.<storage-account>.dfs.core.windows.net", service_credential)
##### spark.conf.set("fs.azure.account.oauth2.client.endpoint.<storage-account>.dfs.core.windows.net", "https://login.microsoftonline.com/<directory-id>/oauth2/token")

# Databricks Utilities
### dbutils.fs() --> to check the file locations

### dbutils.widgets --> allow user to give the input 
#### ex:- dbutils.widgets.text(p_name, 'Ashutosh')

### dbutils.secrets --> to hide the creds in key vault
### dbutils.secrets.list(scope=scope_name) --> it will give you the list of secrets in keyvault
### dbutils.secrets.get(scope=scope_name, key='key_name') --> this will return the secrets of a key to use in Databricks.

# Data Reading

In [0]:
df = spark.read.format('csv')\
    .option('header',True)\
            .option('inferSchema',True)\ #--> spark ready some cols from our data and decide the schema for the df
                .load('URL')

# Delta Lake

In [0]:
# source to destination
df.write.format('delta')\
    .mode('append')\ #--> append, overwrite, error, ignore
        .option('path','url')\
            .save()

#### manage delat table --> it stores meta data in meta store and actual data in cloud storage(default like, data lake). if u delete the metastore metadata then Databricks will delete the acutal data inside the cloud storage.



In [0]:
%sql
CREATE TABLE database_name.table_name
(
  id INT,
  name STRING,
  marks INT
)
USING DELTA

#### External delta table --> it stores meta data in meta store and actual dat ain our cloud storage( our cloud storage). if u delete the metastore metadata then databricks will not delete the actual data inside our cloud storage.

In [0]:
%sql
CREATE TABLE database_name.table_name
(
  id INT,
  name STRING,
  marks INT
)
USING DELTA 
LOCATION 'path'

## Delta table functionalities

#### Once you added the data into the table it will fetch the data from the '_delta_log' folder. it will read the data by reading the json file one by one then the partitions.

#### While performing deletion it will not delete the data from the partitions, instead of that it will create a new partition by remove the particular data. **it is called tombstoning**

# Data Versioning


#### It means that all the crud steps are cosidered as versioning. 

### How to check the history of a table ?

In [0]:
%sql
DESCRIBE HISTORY db_name.table_name

# Rollback or time travel

In [0]:
%sql
RESTORE TABLE Db_name.Table_name TO VERSION AS OF version_number

## To delete the delta table data permanently

In [0]:
#1. vaccum for default time(7days old files are eligible for delete, this is the default time)

%sql
vaccum db_name.table_name

In [0]:
#2. you need to delete the data right now then 
%sql
vaccum db_name.table_name RETAIN 0 HOURS

# Optimize Delta Table

### 1. Optimize

In [0]:
%sql
OPTIMIZE db_name.table_name

## 2. ZORDER BY

In [0]:
%sql
OPTIMIZE db_name.table_name ZORDER BY (col_name)

## workflows are used for the orchestration